Install GRASSGIS, Rasterio, grass-session in order to run GRASS in jupternotebook Python Environment.

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev

In [ ]:
!grass --version

In [ ]:
!pip install rasterio

In [ ]:
!pip install grass-session

In [ ]:
from grass_session import Session

In [ ]:
# Import standard Python packages we need
import sys
import subprocess

# Ask GRASS where its Python packages are to be able to run it from the notebook
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

Warp/Crop [GEDTM30](https://github.com/openlandmap/GEDTM30) into small area. We sent a single URL which points to a remote file (~300GB). With lazy loading, we are able to load small chunk and save to local.

In [ ]:
import rasterio
from rasterio.windows import from_bounds

gedtm30='https://s3.opengeohub.org/global/edtm/gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v1.2.tif'
tmp_dtm_rls_file='gedtm30_tw.tif'
# Bounding box (minx, miny, maxx, maxy)
bbox = (119.25, 21.53, 121.48, 23.23)

with rasterio.open(gedtm30) as src:
    window = from_bounds(*bbox, src.transform)
    out_image = src.read(window=window)
    out_transform = src.window_transform(window)
    out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

with rasterio.open("gedtm30_tw.tif", "w", **out_meta) as dest:
    dest.write(out_image)

[GRASS GIS (8.4.1 Released)](https://grass.osgeo.org/news/2025_03_02_grass_gis_8_4_1_released/) in a nutshell is a computational engine for raster, vector, and geospatial processing.

In [ ]:
# Import the GRASS packages
import grass.script as gs
import grass.jupyter as gj
gs.create_project("OGH2025")

In [ ]:
import time
import shutil
import os
tmp_path='/tmp/grassdata'
start_time = time.time()
# 🚨 delete old location if it exists
if os.path.exists(tmp_path):
    shutil.rmtree(tmp_path)

tmp_twi_file  = tmp_dtm_rls_file.replace('.tif','_twi.tif')
## INPUT DEM for grass
# 1. Start a location in EPSG:4326
with Session(gisdb=tmp_path, location="temp", create_opts="EPSG:4326"):
    start_time = time.time()
    gs.run_command("r.in.gdal", flags='o',overwrite=True,
                   input=tmp_dtm_rls_file,
                   output=f"dmDLSM_30")
    gs.run_command("g.region",
                   raster=f"dmDLSM_30",
                   flags='p')
    print(f"read raster--- %s seconds ---" % (time.time() - start_time))

    start_time = time.time()
    # recale
    gs.mapcalc("mDLSM_30  = dmDLSM_30 * 0.1", overwrite=True)
    print(f"rescale--- %s seconds ---" % (time.time() - start_time))

    start_time = time.time()
    # apply guassian filter
    ## weight factor: sigma factor of gaussian
    gs.run_command("r.neighbors", input="mDLSM_30", output="dem_filtered",
                   weighting_function="gaussian", overwrite=True,
                   weighting_factor=1 )
    print(f"gaussian filter--- %s seconds ---" % (time.time() - start_time))

    # fill the depression
    gs.run_command("r.fill.dir", input="dem_filtered", output="dem_filled", direction="flowdir", overwrite=True)
    print(f"fill depression--- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    # Compute Topographic Wetness Index with r.watershed
    gs.run_command("r.watershed", elevation="dem_filled", tci='twi', overwrite=True)

    gs.run_command('r.out.gdal',overwrite=True,
                    input='twi',
                    output=tmp_twi_file)
    print(f"topographic wetness index--- %s seconds ---" % (time.time() - start_time))